# COMPARATIVE ROAD NETWORK ANALYSIS USING NEO4J

## TASK 1: Listing Roads from/to a Specific City with Distances and Destinations

In [70]:
from neo4j import GraphDatabase
import pandas as pd

uri = "bolt://localhost:7687"                                                             # Neo4j connection details
username = "neo4j"
password = "#Jzee2019"

driver = GraphDatabase.driver(uri, auth=(username, password))                             # Initialize the Neo4j driver

In [71]:
def get_roads_for_city(driver, city_name):
    query = """
    MATCH (c1:City {name: $city_name})-[r:ROAD]->(c2:City)                                
    RETURN c1.name AS From, c2.name AS To, r.distance AS Distance
    UNION
    MATCH (c1:City)-[r:ROAD]->(c2:City {name: $city_name})
    RETURN c1.name AS From, c2.name AS To, r.distance AS Distance
    LIMIT 10;
    """
    with driver.session() as session:
        result = session.run(query, city_name=city_name)
        return [{"FromCity": record["From"], "ToCity": record["To"], "Distance": record["Distance"]} for record in result]

# Retrieve and display the results
city_name = "Atlanta"
roads = get_roads_for_city(driver, city_name)
import pandas as pd
roads_df = pd.DataFrame(roads)
roads_df


,FromCity,ToCity,Distance
0,Atlanta,Santa Ana,139
1,Atlanta,Ankeny,920
2,Atlanta,Paramount,471
3,Atlanta,Fullerton,468
4,Atlanta,Rogers,493
5,Atlanta,Billings,201
6,Atlanta,Tallahassee,764
7,Atlanta,Minneapolis,819
8,Atlanta,North Richland Hills,519
9,Burbank,Atlanta,673


- **Data Representation:** The table lists cities (FromCity and ToCity) and the distances between them (Distance in miles). Each row represents a unique road connection or path between two cities. The query matched outgoing and incoming roads to and from Atlanta and a corresponding distance (in no particular order).  For instance, a road from Atlanta to Santa Ana is 139 units long and one from Bayonne to Atlanta has 413 units long.
  
- **Relevance to the Task:** The data showcases a small sample of a larger road network dataset. It demonstrates the relationship between cities and their connectivity, which is foundational for building a graph in Neo4j with cities as nodes and roads (with distances) as edges.
  
- **Insight for Import:** The data structure (columns FromCity, ToCity, and Distance) is correctly formatted for Neo4j batch processing, enabling the creation of relationships (ROAD) between nodes (City). This verifies that the dataset is suitable for importing into a graph database to analyze or visualize road connectivity.
 

## TASK 2: Finding Roads Longer than a 400 miles

In [166]:
def find_long_roads(min_distance):
    # Cypher query to find roads longer than the specified minimum distance
    # Limits the returned results to 10, but counts the total that match the criteria
    query = """
    MATCH (city1:City)-[r:ROAD]->(city2:City)
    WHERE r.distance > $min_distance
    RETURN city1.name AS FromCity, city2.name AS ToCity, r.distance AS Distance
    ORDER BY r.distance DESC
    LIMIT 10
    """
    
    count_query = """
    MATCH (city1:City)-[r:ROAD]->(city2:City)
    WHERE r.distance > $min_distance
    RETURN COUNT(r) AS TotalMatchingRoads
    """
    
    with driver.session() as session:
        result = session.run(query, min_distance=min_distance)                                          # Fetching the top 10 results
        roads = result.data()
        count_result = session.run(count_query, min_distance=min_distance)                              # Fetching the count of all roads meeting the criteria
        total_matching = count_result.single()["TotalMatchingRoads"]
        print(f"Total roads longer than {min_distance} miles: {total_matching}")                           # Printing the total count of matching roads
        
        for road in roads:                                                                              # Printing the details of the first 10 matching roads
            print(f"From: {road['FromCity']}, To: {road['ToCity']}, Distance: {road['Distance']} miles")

find_long_roads(400)                                                                                    # Call function for roads longer than 100 km

Total roads longer than 400 miles: 1997
From: Fort Worth, To: Milford, Distance: 1000 miles
From: South Bend, To: Houston, Distance: 1000 miles
From: Lexington, To: Dayton, Distance: 1000 miles
From: Bayonne, To: Rowlett, Distance: 1000 miles
From: Alameda, To: Pico Rivera, Distance: 999 miles
From: Arlington, To: Palm Beach Gardens, Distance: 999 miles
From: Plymouth, To: Kirkland, Distance: 999 miles
From: Altamonte Springs, To: Oak Lawn, Distance: 998 miles
From: Raleigh, To: St. Cloud, Distance: 998 miles
From: Fountain Valley, To: Fort Lee, Distance: 997 miles


- **What the Code Did:** The code queried the Neo4j graph for roads with a distance greater than 400 km. We opted for 400km for meaningful results as by default our dataset distance starts at 100 so all observations would have met the criteria thus not meaningless for analysis. The code fetched the top 10 longest roads and counted how many roads met the criteria.

- **Total Count:** The result shows there are 1997 roads longer than 400 km, consistent with the raw data and attached Cypher output after ensuring only distinct edges (avoiding duplicates in both directions).

- **Top 10 Roads:** The output displays the FromCity, ToCity, and Distance for the top 10 longest roads, with distances ranging from 997 km to 1000 km. For example, the longest road is between Fort Worth and Milford, with a distance of 1000 km.

## TASK 3: Number of Roads Connected to a Specific City (Washington, Phoenix, North Miami)

In [77]:
def count_roads_for_city(city_name):
    query = """
    MATCH (city:City {name: $city_name})-[r:ROAD]-(connected_city:City)
    RETURN COUNT(r) AS RoadCount
    """
    with driver.session() as session:
        result = session.run(query, city_name=city_name)
        road_count = result.single()["RoadCount"]
        print(f"Total roads connected to {city_name}: {road_count}")

                                                                                          # Counting roads for Washington, Phoenix, and North Miami
count_roads_for_city("Washington")
count_roads_for_city("Phoenix")
count_roads_for_city("North Miami")


Total roads connected to Washington: 32
Total roads connected to Phoenix: 24
Total roads connected to North Miami: 32


- **Why count both directions:** Roads (or connections) to a city can be both incoming (roads leading to the city) and outgoing (roads leaving the city). Both types of connections represent valid relationships in the network.
  
- **How the code worked:** The code used Cypher queries to count the total number of road connections (both incoming and outgoing) for specific cities (Washington, Phoenix, and North Miami) in a graph database. It matched the cities to their connected roads, counting the relationships (edges) that linked them to other cities in the network, regardless of direction.
  
- **Output**: The results showed the following number of total connections: 32 for Washington, 24 for Phoenix, and 32 for North Miami, indicating the total roads connecting each city to others in the dataset.

## TASK 4: Finding Cities Connected to Memphis with a Distance Below 300km

In [168]:
def find_nearby_cities(city_name, max_distance):
    query = """
    MATCH (city:City {name: $city_name})-[r:ROAD]->(connected_city:City)
    WHERE r.distance < $max_distance
    RETURN connected_city.name AS ConnectedCity, r.distance AS Distance
    ORDER BY r.distance
    """
    
    with driver.session() as session:
        result = session.run(query, city_name=city_name, max_distance=max_distance)
        for record in result:
            print(f"Connected City: {record['ConnectedCity']}, Distance: {record['Distance']} miles")

find_nearby_cities("Memphis", 300)                                                   # Finding cities connected to Memphis with roads shorter than 300 km

Connected City: Apple Valley, Distance: 126 miles
Connected City: Palatine, Distance: 164 miles
Connected City: San Ramon, Distance: 172 miles
Connected City: Sanford, Distance: 236 miles
Connected City: Boise, Distance: 239 miles
Connected City: Mansfield, Distance: 299 miles
Connected City: Bozeman, Distance: 299 miles


- **Connected Cities:** The output lists cities directly connected to a specific city (e.g., San Francisco), where the connection is made via a road that has a distance less than 300 km. The cities are shown along with the respective distance in kilometers.

- **Output**: This code allows you to query and display specific connections between cities based on the road distance, which is useful for analyzing proximity or building network maps between locations. For instance connected city Apple Valley has the shortest distance from Memphis at 126km hence in close proximity ahead of other cities. The other cities listed (San Ramon, Sanford, Boise, Mansfield, Bozeman) are similarly connected by roads shorter than 300 km.

## TASK 5: Listing All Roads Leading To Los Angeles.

In [174]:
def list_roads_to_city(target_city):
    query = """
    MATCH (city:City)-[r:ROAD]->(target_city:City {name: $target_city})
    RETURN city.name AS FromCity, r.distance AS Distance
    ORDER BY r.distance DESC
    """
    with driver.session() as session:
        result = session.run(query, target_city=target_city)
        roads = result.data()
        for road in roads:
            print(f"From: {road['FromCity']}, Distance: {road['Distance']} miles")

list_roads_to_city('Los Angeles')                                                      # Listing all roads leading to Los Angeles

From: Bayonne, Distance: 996 miles
From: Las Vegas, Distance: 944 miles
From: Bartlett, Distance: 828 miles
From: South Bend, Distance: 284 miles
From: Euless, Distance: 138 miles
From: Severn, Distance: 109 miles


- **Condition**: The query matches all cities (city) that have a ROAD relationship pointing to "Los Angeles" (target city). The RETURN clause displays the names of cities that lead to Los Angeles, along with the distances of the roads.The results are ordered by the distance of the road in descending order (ORDER BY r.distance DESC).

- Bayonne is connected to Los Angeles by a road that is 996 km long and Severn (109 km) is the shortest connection listed. This type of query can be useful for visualizing transportation networks or understanding city connectivity flows into Los Angeles. It shows how cities are flows into LA, including both close and long-distance routes.

## TASK 6: The Total Distance Of All Roads Connected to Memphis.

In [176]:
def calculate_total_distance_and_connections(city_name):
    query = """
    MATCH (city:City {name: $city_name})-[r:ROAD]-(connected_city:City)
    RETURN SUM(r.distance) AS TotalDistance, COUNT(r) AS TotalConnections
    """
    with driver.session() as session:
        result = session.run(query, city_name=city_name)
        record = result.single()
        total_distance = record["TotalDistance"]
        total_connections = record["TotalConnections"]
        print(f"Total Distance of all roads connected to {city_name}: {total_distance} miles")
        print(f"Total number of connections to {city_name}: {total_connections}")

calculate_total_distance_and_connections("Memphis")

Total Distance of all roads connected to Memphis: 17832 miles
Total number of connections to Memphis: 34


- **Total Distance of all roads connected to Memphis: 17832 km:** This result represents the total length of all roads (in kilometers) that are directly connected to the city of Memphis. The roads may lead to or come from Memphis, encompassing all connecting paths.

- **Total number of connections to Memphis: 34:** This number indicates how many direct road connections (edges) Memphis has with other cities in the database. It counts each unique road link, regardless of whether the road leads to Memphis or originates from it.

- These results answer the task by quantifying the connectivity of Memphis, both in terms of total distance and number of roads connecting the city to others. This is useful for understanding Memphis's role in the broader network of cities and its accessibility via road infrastructure.

## TASK7: Shortest Road Between Las Vegas and Houston

In [178]:
def find_shortest_road(city1, city2):
    query = """
    MATCH (city1:City {name: $city1})-[r:ROAD]-(city2:City {name: $city2})
    RETURN city1.name AS FromCity, city2.name AS ToCity, MIN(r.distance) AS ShortestDistance
    """
    with driver.session() as session:
        result = session.run(query, city1=city1, city2=city2)
        road = result.single()
        if road:
            print(f"Shortest road from {road['FromCity']} to {road['ToCity']} is {road['ShortestDistance']} miles")

find_shortest_road("Las Vegas", "Houston")

Shortest road from Las Vegas to Houston is 370 miles


- **Shortest Road:** This output displays the shortest road between Las Vegas and Houston as 370km, showing both the starting city ("FromCity"), destination city ("ToCity"), and the minimum distance (in kilometers). This approach ensures you get the minimal distance among all direct roads between the two cities, which is essential for optimizing travel routes or infrastructure planning.

## TASK8: Cities Directly Connected to Both Philadelphia and Seattle

In [92]:
def find_cities_connected_to_both(city1, city2):
    query = """
    MATCH (city1:City {name: $city1})-[:ROAD]-(connected_city:City)-[:ROAD]-(city2:City {name: $city2})
    RETURN connected_city.name AS ConnectedCity
    """
    with driver.session() as session:
        result = session.run(query, city1=city1, city2=city2)
        connected_cities = result.data()
        for city in connected_cities:
            print(f"Connected city: {city['ConnectedCity']}")

find_cities_connected_to_both("Philadelphia", "Seattle")

Connected city: Fargo
Connected city: Scottsdale
Connected city: Mansfield
Connected city: Gainesville
Connected city: Bentonville
Connected city: Palmdale
Connected city: Mesa
Connected city: Marietta
Connected city: Anaheim


- The cities above serve as intersections in the network of roads that link Philadelphia and Seattle. The query in Cypher successfully identified all the cities that share roads with both Philadelphia and Seattle, making it useful for analyzing transportation routes or connectivity within a network. In essence, the query ensures you identify cities that play a central role in connecting different regions, making it an essential tool for transportation or network analysis.

## TASK9: Cities with More Than 3 Direct Connections and Their Connections

In [95]:
def get_cities_with_multiple_connections():
    query = """
    MATCH (city:City)-[:ROAD]-(connected_city:City)
    WITH city, COLLECT(connected_city) AS connections
    WHERE SIZE(connections) > 3
    RETURN city.name AS City, [connected IN connections | connected.name] AS Connections, SIZE(connections) AS num_connections
    ORDER BY num_connections DESC
    LIMIT 10
    """
    
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            print(f"City: {record['City']}")
            print(f"Connections: {', '.join(record['Connections'])}")
            print(f"Total Connections: {record['num_connections']}")
            print("-" * 30)
            
get_cities_with_multiple_connections()

City: Arlington
Connections: Kansas City, Long Beach, Cedar Hill, Phoenix, Dearborn, Bristol, Baldwin Park, Gainesville, Washington, Lehi, Little Elm, Henderson, Round Rock, Paramount, Palatine, Victoria, Simi Valley, Bakersfield, Daly City, San Bruno, Goodyear, Lawrence, Tulsa, Blaine, West Sacramento, Dubuque, Palm Beach Gardens, North Las Vegas, St. Peters, Fort Collins, Plymouth, South Bend, Los Angeles, San Leandro, Fort Worth, Fargo, Scottsdale, St. Louis, Newark, Rancho Palos Verdes, Greeley, Sterling Heights, St. George, Apple Valley, Roseville, Pomona, Eden Prairie, Rowlett, DeKalb
Total Connections: 49
------------------------------
City: Mission Viejo
Connections: Hayward, Santa Ana, Lacey, Springfield, Long Beach, Oak Lawn, Pico Rivera, Columbia, Hagerstown, Escondido, Atlanta, Dayton, Baldwin Park, Irving, Fremont, Madison, Keller, Bell Gardens, Baton Rouge, Mobile, Henderson, Centennial, White Plains, Schertz, Simi Valley, Reno, Toledo, Lawrence, Chino, Mesa, Tallahassee,

- The query first matches cities and their connected cities with roads. It then counts the number of direct connections for each city. Cities with more than 3 connections are filtered. The COLLECT() function is used to gather all connected cities for each city that meets the criteria. The results are returned in descending order of the number of connections.

- **LIMIT 10:** The LIMIT 10 at the end of the Cypher query ensures that only the top 10 cities with the most connections (more than 3) are returned. Python Execution: The Python code will execute the query and print out the results for the top 10 cities, listing each city’s name, its connected cities, and the total number of connections.

## TASK10: Total Distance Of All Connections In The Network

In [180]:
def calculate_total_distance():
    query = """
    MATCH (:City)-[r:ROAD]->(:City)
    RETURN SUM(r.distance) AS Total_Distance
    """
    
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            print(f"Total Distance of All Roads: {record['Total_Distance']} miles")

calculate_total_distance()

Total Distance of All Roads: 1644013 miles


- The query matches all ROAD relationships between cities, regardless of the direction. It then aggregates (SUM()) the distance property of each ROAD relationship. The result is the total distance of all connections in the network.

- The output, **"Total Distance of All Roads: 1,644,013 km",** represents the cumulative distance of all road connections in the dataset. It is the sum of all distances between cities connected by roads in the graph database, calculated by aggregating the distance property of all ROAD relationships. This total is an indicator of the overall road network's size, measured in kilometers.

# EXTRA CREDIT QUESTIONS

## 1. Identifying Cities with Exactly Two Connections

In [124]:
def find_cities_with_two_connections(driver):
    query = """
    MATCH (c:City)
    OPTIONAL MATCH (c)-[:ROAD]->(outgoing:City)
    OPTIONAL MATCH (incoming:City)-[:ROAD]->(c)
    WITH c, 
         COLLECT(DISTINCT outgoing.name) AS OutgoingCities, 
         COLLECT(DISTINCT incoming.name) AS IncomingCities
    WHERE SIZE(OutgoingCities) + SIZE(IncomingCities) = 2
    RETURN c.name AS City, OutgoingCities, IncomingCities
    """
    with driver.session() as session:
        result = session.run(query)
        records_found = False
        for record in result:
            records_found = True
            print(f"City: {record['City']}")
            print(f"Outgoing Cities: {record['OutgoingCities']}")
            print(f"Incoming Cities: {record['IncomingCities']}\n")
        if not records_found:
            print("No cities with exactly two connections were found.")

- **The code identifies** cities with exactly two direct connections (one incoming and one outgoing, or two of the same type). It aggregates connected cities using COLLECT() and filters cities where the total number of connections equals 2. The final results display the city name and its two connected cities.

- **Reason for No Output**
The dataset configuration ensures every city has at least 15 connections, so no city satisfies the condition of having exactly two connections. Since Neo4j's RETURN statement does not find any matches, it outputs nothing rather than a zero or placeholder value. Neo4j only returns records that match the query conditions. If no matches exist, the Python result iterator contains no elements, so the for loop does not execute, and no output is printed.

In [139]:
def find_cities_with_two_connections():
    query = """
    MATCH (c:City)-[r:ROAD]->()
    WITH c, COUNT(*) AS outgoingConnections
    WHERE outgoingConnections = 2
    RETURN c.name AS City, outgoingConnections
    """
    
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            print(f"City: {record['City']}, Outgoing Connections: {record['outgoingConnections']}")

find_cities_with_two_connections()


City: Miramar, Outgoing Connections: 2
City: West Haven, Outgoing Connections: 2
City: Spokane, Outgoing Connections: 2
City: North Las Vegas, Outgoing Connections: 2


In [143]:
def find_cities_with_two_incoming_connections():
    query = """
    MATCH (c:City)<-[r:ROAD]-()
    WITH c, COUNT(*) AS incomingConnections
    WHERE incomingConnections = 2
    RETURN c.name AS City, incomingConnections
    """
    
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            print(f"City: {record['City']}, Incoming Connections: {record['incomingConnections']}")

find_cities_with_two_incoming_connections()

City: St. Louis, Incoming Connections: 2
City: Las Vegas, Incoming Connections: 2


- Although we don't have cities with exactly 2 total connections (combined incoming and outgoing or total either or). We do however have cities with 2 outgoing connections and 2 incoming connections as shown on 2 outputs above.

- These results reflect the cities in the dataset that have 2 outgoing or incoming connections. The absence of cities with exactly 2 total connections (whether incoming, outgoing, or both) is due to the dataset configuration, where most cities have at least 15 connections.

## 2. Cities with Both Incoming and Outgoing Connections

In [159]:
import warnings
import logging

warnings.filterwarnings("ignore", message=".*AggregationSkippedNull.*")
logging.basicConfig(level=logging.ERROR)

def find_cities_with_incoming_and_outgoing_connections():
    query = """
    MATCH (c:City)
    OPTIONAL MATCH (c)-[r1:ROAD]->()  // Outgoing roads
    OPTIONAL MATCH (c)<-[r2:ROAD]-()  // Incoming roads
    WITH c, 
         COUNT(DISTINCT r1) AS outgoingConnections, 
         COUNT(DISTINCT r2) AS incomingConnections
    WHERE outgoingConnections > 0 AND incomingConnections > 0
    RETURN c.name AS City, outgoingConnections, incomingConnections
    ORDER BY outgoingConnections + incomingConnections DESC
    LIMIT 10
    """
    
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            print(f"City: {record['City']}, Outgoing Connections: {record['outgoingConnections']}, Incoming Connections: {record['incomingConnections']}")

find_cities_with_incoming_and_outgoing_connections()

City: Arlington, Outgoing Connections: 28, Incoming Connections: 21
City: Mission Viejo, Outgoing Connections: 33, Incoming Connections: 14
City: Lexington, Outgoing Connections: 39, Incoming Connections: 6
City: Fort Worth, Outgoing Connections: 33, Incoming Connections: 10
City: Henderson, Outgoing Connections: 1, Incoming Connections: 41
City: Paramount, Outgoing Connections: 9, Incoming Connections: 33
City: North Lauderdale, Outgoing Connections: 37, Incoming Connections: 4
City: Maricopa, Outgoing Connections: 25, Incoming Connections: 16
City: Apple Valley, Outgoing Connections: 34, Incoming Connections: 6
City: Kokomo, Outgoing Connections: 19, Incoming Connections: 20


- **COUNT(DISTINCT r1) and COUNT(DISTINCT r2):** This ensures that we are counting distinct roads for outgoing and incoming connections. If the same road is counted both as outgoing and incoming for the same city, it won't be double-counted. **OPTIONAL MATCH:** We still use OPTIONAL MATCH to include cities with no connections in either direction (in case they exist). **WHERE outgoingConnections > 0 AND incomingConnections > 0:** This ensures that only cities with at least one incoming and one outgoing connection are included in the results.
  
- **We returned cities with distinct incoming and outgoing connections.** The number of outgoing and incoming connections for each city will reflected different sets of connections, rather than being equal unless the city's connections are symmetric. **LIMT 10**

## 3. Top 3 Pairs of Cities with the Longest Direct Distances

In [164]:
def find_top_3_longest_connections():
    query = """
    MATCH (start:City)-[r:ROAD]->(end:City)
    RETURN start.name AS StartCity, end.name AS EndCity, r.distance AS Distance
    ORDER BY r.distance DESC
    LIMIT 3
    """
    
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            print(f"Start City: {record['StartCity']}, End City: {record['EndCity']}, Distance: {record['Distance']} miles")

find_top_3_longest_connections()

Start City: Lexington, End City: Dayton, Distance: 1000 miles
Start City: South Bend, End City: Houston, Distance: 1000 miles
Start City: Bayonne, End City: Rowlett, Distance: 1000 miles


- This code **matches all roads connecting cities**. It assumes that the City nodes are connected by ROAD relationships, with **r.distance being the road's distance between the cities**. Then returns the names of the start and end cities along with the distance between them. Then orders the results by the distance, from the longest to the shortest.
- The output limits the result to the top 3 city pairs with the longest distances as shown in the output. This output can also be verified with output from Task 2 where we listed cities longer than 400 miles, lexington to Dayton, South Bend to Houston, and Bayonne to Rowlett are longest direct distances with 1000 miles

# OVERALL ANALYSIS AND OBSERVATIONS

- **WE HAVE ATTACHED A SEPERATE FILE WITH OUR OVERALL ANALYSIS AND OBSERSATIONS FOR YOU EASE ACCESS**